# Application in Lung Tumour Application

In this application, we follow the following steps:
1. Convert MRIs to binary masks, volumes and sample point cloud from the tumour surface
2. Construct bi-parameter filtration and compute corresponding rank invariant
3. Classification using KNN and MBD

In [1]:
# load modeule
import pylidc as pl
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import csv
# set working directory
workdir = "/home/qw817/Desktop/Lung_TDA_application"
import os
import sys
from os import listdir
from os.path import isfile, join
os.chdir(workdir)
sys.path.insert(0, os.path.join(workdir, "Functions"))

# Handling dicom images
import SimpleITK as sitk

# Persistent homology and topological feature extraction
import TDAfeatures as tf
from ripser import ripser
from persim import plot_diagrams

# training classifier
import skfda
from skfda.ml.classification import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from skfda.representation.grid import FDataGrid
from sklearn.metrics import roc_auc_score
import statistics
from skfda.exploratory.depth import ModifiedBandDepth
from skfda.ml.classification import MaximumDepthClassifier

In [5]:
# import diagnosis
df = pd.read_csv('Diagnosis.csv', sep = ',') 
diagnosis = np.asarray(df) 
full_classes = diagnosis[:,1]

# Step 1: Convert MRIs into samples of points from the tumour surface

In [6]:
ids = []
sizes = []
num = 0
all_masks = []
all_pc =[]
for i in diagnosis[:,0]:
    # if annotation available generate binary masks and point clouds
    if len(pl.query(pl.Scan).filter(pl.Scan.patient_id == i).first().annotations)!=0:
        # identify annotation
        ann = pl.query(pl.Annotation).join(pl.Scan)\
                .filter(pl.Scan.patient_id == i).first()
        # generate binary mask
        mask = ann.boolean_mask()
        all_masks.append(mask)
        # concate mask into one large matrix
        test1 = np.concatenate([mask[:,:,j] for j in range(mask.shape[2])], axis=0)
        # save np array as csv
        # np.savetxt("masks/"+i+"_mask.csv", test1, delimiter=",")
        # save id
        ids.append(i)
        # save mask shape
        sizes.append(mask.shape)

        # generate pc
        vol = tf.read_dcm("dcm/"+i[-4:])
        verts, faces = tf.mesh_from_lesion(vol, mask)
        all_pc.append(verts)
        # save point clouds
        #np.savetxt("point_clouds/"+i+"_pc.csv", verts, delimiter = ",")

In [7]:
# subsample for point clouds larger than 1000
small_pc =[]
for i in range(len(ids)):
    current = all_pc[i]
    if current.shape[0]>1000:
        # randomly shuffle data
        np.random.shuffle(current)
        current = current[:1000,:]
        small_pc.append(current)
    else:
        small_pc.append(current)
    # save point clouds
    # np.savetxt("small_point_clouds/pc_"+ids[i][-4:]+".txt", current, delimiter=',')

# Step 2: Construct bi-parameter filtration and corresponding rank invariant
The bi-filtrations include:
- Degree-rips filtration
- H-rips filtration
- X-rips filtration
- Y-rips filtration

In [ ]:
# first can compute rank invariants with degree-rips filtration without adding new filtration values to the data
degree_rips_rank_invariants = []
for i in ids:
    # compute bi-parameter using rivet    
    cmd = "/home/qw817/rivet/rivet_console /home/qw817/Desktop/Lung_TDA_application/small_point_clouds/pc_"+i[-4:]+".txt /home/qw817/Desktop/Lung_TDA_application/rivet_outputs/rivet"+i[-4:]+".rivet --datatype points --homology 0 --xbins 20 --ybins 20"
    os.popen(cmd).read().split('\n')
    # convert the rivert file to rank invariants
    with open("rivet_outputs/rivet"+i[-4:]+".rivet", 'rb') as f:
        computed_data = f.read()
    degree_rips_rank_invariants.append(rank_function(computed_data, grid_size=20, fixed_bounds=None, use_weights=False, normalize=False, minimum_rank=0))
# convert to and save as matrix
degree_rips_rank_invariants_mat = np.array(degree_rips_rank_invariants)
# np.savetxt("computed_rank_invariants_degree_rips.csv", np.array(degree_rips_rank_invariants), delimiter=',')

### The other three filtrations requires construction

In [ ]:
# include second filtration - height/z parameter
n = len(ids)
for i in range(n):
    pid = ids[i]
    ppc = small_pc[i]
    
    # write to file
    file_name = "small_point_cloud_height_filtration/h_ptcl_"+pid[-4:]+".txt"
    f = open(file_name, "w")
    f.write("--datatype points_fn\n")
    f.write("\n")
    f.write(",".join(map(str, ppc[:,2])))
    f.write("\n")

    for j in range(ppc.shape[0]):
        f.write("\n")
        f.write(",".join(map(str, ppc[j,:])))
    
    f.close()  

In [ ]:
# compute rivet file and rank invariant
h_rips_rank_invariants = []
for i in ids:
    # compute rivet file
    cmd = "/home/qw817/rivet/rivet_console /home/qw817/Desktop/Lung_TDA_application/small_point_cloud_height_filtration/h_ptcl_"+i[-4:]+".txt /home/qw817/Desktop/Lung_TDA_application/height_rivet/h_pc"+i[-4:]+".rivet --xbins 20 --ybins 20"
    os.popen(cmd).read().split('\n')
    # convert the rivert file to rank invariants
    with open("height_rivet/h_pc"+i[-4:]+".rivet", 'rb') as f:
        computed_data = f.read()
    h_rips_rank_invariants.append(rank_function(computed_data, grid_size=20, fixed_bounds=None, use_weights=False, normalize=False, minimum_rank=0))
# convert to and save as matrix
h_rips_rank_invariants_mat = np.array(h_rips_rank_invariants)
# np.savetxt("computed_rank_invariants_height.csv", np.array(h_rips_rank_invariants), delimiter=',')    

In [ ]:
# include second filtration - x parameter
n = len(ids)
for i in range(n):
    pid = ids[i]
    ppc = small_pc[i]
    
    # write to file
    file_name = "x_pc/pc_"+pid[-4:]+".txt"
    f = open(file_name, "w")
    f.write("--datatype points_fn\n")
    f.write("\n")
    f.write(",".join(map(str, ppc[:,0])))
    f.write("\n")

    for j in range(ppc.shape[0]):
        f.write("\n")
        f.write(",".join(map(str, ppc[j,:])))
    
    f.close()  

In [ ]:
# compute rivet file and rank invariant
x_rips_rank_invariants = []
for i in ids:
    # compute rivet file
    cmd = "/home/qw817/rivet/rivet_console /home/qw817/Desktop/Lung_TDA_application/x_pc/pc_"+i[-4:]+".txt /home/qw817/Desktop/Lung_TDA_application/x_rivet/x_pc"+i[-4:]+".rivet --xbins 20 --ybins 20"
    os.popen(cmd).read().split('\n')
    # convert the rivert file to rank invariants
    with open("height_rivet/x_pc"+i[-4:]+".rivet", 'rb') as f:
        computed_data = f.read()
    x_rips_rank_invariants.append(rank_function(computed_data, grid_size=20, fixed_bounds=None, use_weights=False, normalize=False, minimum_rank=0))
# convert to and save as matrix
x_rips_rank_invariants_mat = np.array(x_rips_rank_invariants)
# np.savetxt("computed_rank_invariants_x.csv", np.array(x_rips_rank_invariants), delimiter=',') 

In [ ]:
# include second filtration - y parameter
n = len(ids)
for i in range(n):
    pid = ids[i]
    ppc = small_pc[i]
    
    # write to file
    file_name = "y_pc/pc_"+pid[-4:]+".txt"
    f = open(file_name, "w")
    f.write("--datatype points_fn\n")
    f.write("\n")
    f.write(",".join(map(str, ppc[:,1])))
    f.write("\n")

    for j in range(ppc.shape[0]):
        f.write("\n")
        f.write(",".join(map(str, ppc[j,:])))
    
    f.close()    

In [ ]:
# compute rivet file and rank invariant
y_rips_rank_invariants = []
for i in ids:
    # compute rivet file
    cmd = "/home/qw817/rivet/rivet_console /home/qw817/Desktop/Lung_TDA_application/y_pc/pc_"+i[-4:]+".txt /home/qw817/Desktop/Lung_TDA_application/y_rivet/y_pc"+i[-4:]+".rivet --xbins 20 --ybins 20"
    os.popen(cmd).read().split('\n')
    # convert the rivert file to rank invariants
    with open("height_rivet/y_pc"+i[-4:]+".rivet", 'rb') as f:
        computed_data = f.read()
    y_rips_rank_invariants.append(rank_function(computed_data, grid_size=20, fixed_bounds=None, use_weights=False, normalize=False, minimum_rank=0))
# convert to and save as matrix
y_rips_rank_invariants_mat = np.array(y_rips_rank_invariants)
# np.savetxt("computed_rank_invariants_y.csv", np.array(y_rips_rank_invariants), delimiter=',') 

# Step 3: Train classifiers
- k-NN classifier
- MBD classifer

Import Meta data to find which contains contrasting material

In [20]:
# import meta data
meta = pd.read_csv("Meta.csv")
# contruct a dictionary of whether an image contained contrasting material
contrast_dict = {}
for i in range(meta.shape[0]):
    contrast_dict[meta.to_numpy()[i,0][10:14]]=meta.to_numpy()[i,10]

# find indices for which the tumours are benign or malignant
benign_index = np.where(full_classes == 1)[0]
malignant_index = np.where(full_classes == 2)[0]
# and those with and without contrast
benign_index_contrast = [i for i in benign_index if contrast_dict[ids[i][-4:]]=="Y"]
malignant_index_contrast = [i for i in malignant_index if contrast_dict[ids[i][-4:]]=="Y"]

In [ ]:
# join indexes together
primary_index_wcontrast = benign_index_contrast + malignant_index_contrast
primary_index_no_contrast = benign_index.tolist() + malignant_index.tolist()
# labels
labels_wcontrast = [0 for i in range(len(benign_index_contrast))] + [1 for i in range(len(malignant_index_contrast))]
labels_no_contrast = [0 for i in range(len(benign_index.tolist()))] + [1 for i in range(len(malignant_index.tolist()))]
# contrast ranks
dranks_wc = d_ranks[primary_index_wcontrast,:]
hranks_wc = h_ranks[primary_index_wcontrast,:]
xranks_wc = x_ranks[primary_index_wcontrast,:]
yranks_wc = y_ranks[primary_index_wcontrast,:]

# no contrast ranks
dranks_nc = d_ranks[primary_index_no_contrast,:]
hranks_nc = h_ranks[primary_index_no_contrast,:]
xranks_nc = x_ranks[primary_index_no_contrast,:]
yranks_nc = y_ranks[primary_index_no_contrast,:]

In [ ]:
# define a function that carries out knn over n iterations and outputs:
# average accuracy, all accuracies, average rocauc, all rocauc
def Knn_accuracy(X, y, n_iter, test_ratio=0.25, nn = 5):
    all_acc = [1000 for j in range(n_iter)]
    all_roc = [1000 for j in range(n_iter)]
    for i in range(n_iter):
        X_train, X_test, y_train, y_test = train_test_split(
            FDataGrid(X),
            y.astype('int'),
            test_size = test_ratio,
            stratify=y,
        )
        knn = KNeighborsClassifier(n_neighbors=nn)
        knn.fit(X_train, y_train)
        all_acc[i] = knn.score(X_test, y_test)
        pred = knn.predict(X_test)
        all_roc[i] = roc_auc_score(y_test, pred)

    acc = sum(all_acc)/n_iter
    avg_roc = sum(all_roc)/n_iter
    print("Average accuracy of Knn is ", acc, "And average ROCAUC of Knn is ", avg_roc)
    return acc, all_acc, avg_roc, all_roc

In [ ]:
# train knn classifier on data with contrast
print("Train knn classifier on data with contrast ")

print("performance of k-NN classifier trained on height-rips")
a,b,c,d = Knn_accuracy(hranks_wc, np.array(labels_wcontrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(b),"roc variance ", statistics.stdev(d))


print("performance of k-NN classifier trained on degree-rips")
a,b,c,d = Knn_accuracy(dranks_wc, np.array(labels_wcontrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(b),"roc variance ", statistics.stdev(d))

print("performance of k-NN classifier trained on y-rips")
a,b,c,d = Knn_accuracy(yranks_wc, np.array(labels_wcontrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(b),"roc variance ", statistics.stdev(d))

print("performance of k-NN classifier trained on x-rips")
a,b,c,d = Knn_accuracy(xranks_wc, np.array(labels_wcontrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(b),"roc variance ", statistics.stdev(d))

In [ ]:
# train knn classifier on data without contrast 
print("Train knn classifier on data without contrast ")

print("performance of k-NN classifier trained on height-rips")
a,b,c,d = Knn_accuracy(hranks_nc, np.array(labels_no_contrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(b),"roc variance ", statistics.stdev(d))


print("performance of k-NN classifier trained on degree-rips")
a,b,c,d = Knn_accuracy(dranks_nc, np.array(labels_no_contrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(b),"roc variance ", statistics.stdev(d))

print("performance of k-NN classifier trained on y-rips")
a,b,c,d = Knn_accuracy(yranks_nc, np.array(labels_no_contrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(b),"roc variance ", statistics.stdev(d))

print("performance of k-NN classifier trained on x-rips")
a,b,c,d = Knn_accuracy(xranks_nc, np.array(labels_no_contrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(b),"roc variance ", statistics.stdev(d))


In [ ]:
# Max depth classifier
# define a function that carries out knn over n iterations and outputs the average aucroc, average accuracies, all aucroc and all accuracies
def Maxdepth_acc_roc(X, y, n_iter, test_ratio=0.25, nn = 5):
    all_roc = [1000 for j in range(n_iter)]
    all_acc = [1000 for j in range(n_iter)]
    for i in range(n_iter):
        X_train, X_test, y_train, y_test = train_test_split(
            FDataGrid(X),
            y.astype('int'),
            test_size = test_ratio,
            stratify=y,
        )
        depth = MaximumDepthClassifier()
        depth.fit(X_train, y_train)
        pred = depth.predict(X_test)
        all_roc[i] = roc_auc_score(y_test, pred)
        all_acc[i] = depth.score(X_test, y_test)
    avg = sum(all_roc)/n_iter
    acc = sum(all_acc)/n_iter
    print("Average aucroc of Max Depth classifier is ", avg)
    print("Average accuracy of Max Depth classifier is ", acc)
    return avg,acc, all_roc, all_acc

In [ ]:
print("Train MBD classifier on data with contrast ")

print("performance of MBD classifier trained on height-rips")
a,b,c,d = Maxdepth_acc_roc(hranks_wc, np.array(labels_wcontrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(d),"roc variance ", statistics.stdev(c))


print("performance of MBD classifier trained on height-rips")
a,b,c,d = Maxdepth_acc_roc(dranks_wc, np.array(labels_wcontrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(d),"roc variance ", statistics.stdev(c))

print("performance of MBD classifier trained on y-rips")
a,b,c,d = Maxdepth_acc_roc(yranks_wc, np.array(labels_wcontrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(d),"roc variance ", statistics.stdev(c))

print("performance of MBD classifier trained on x-rips")
a,b,c,d = Maxdepth_acc_roc(xranks_wc, np.array(labels_wcontrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(d),"roc variance ", statistics.stdev(c))


In [ ]:
print("Train MBD classifier on data without contrast ")

print("performance of MBD classifier trained on height-rips")
a,b,c,d = Maxdepth_acc_roc(hranks_nc, np.array(labels_no_contrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(d),"roc variance ", statistics.stdev(c))

print("performance of MBD classifier trained on height-rips")a,b,c,d = Maxdepth_acc_roc(dranks_nc, np.array(labels_no_contrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(d),"roc variance ", statistics.stdev(c))

print("performance of MBD classifier trained on y-rips")
a,b,c,d = Maxdepth_acc_roc(yranks_nc, np.array(labels_no_contrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(d),"roc variance ", statistics.stdev(c))

print("performance of MBD classifier trained on x-rips")
a,b,c,d = Maxdepth_acc_roc(xranks_nc, np.array(labels_no_contrast), 50, test_ratio=0.25, nn=5)
print("accuracy variance ", statistics.stdev(d),"roc variance ", statistics.stdev(c))